# Rule Controller example

First we import all the used libraries, remember to always import `sinergym` even if it says is not used, because that is needed to define the environments

In [1]:
from typing import List, Any, Sequence
from sinergym.utils.common import get_season_comfort_range
from datetime import datetime
import gymnasium as gym
import numpy as np
import sinergym

Now we can define the environment we want to use, in our case we are using the Eplus demo.
Don't forget to deactivate the `flag_normalization`. It is important due to the fact that *MyRuleBasedController*
works with real values, not `[-1,1]` space.

In [3]:
env = gym.make('Eplus-5Zone-hot-continuous-v1', flag_normalization=False)

[2023-06-28 08:54:00,262] EPLUS_ENV_5Zone-hot-continuous-v1_MainThread_ROOT INFO:Updating Building model ExternalInterface object if it is not present...
[2023-06-28 08:54:00,263] EPLUS_ENV_5Zone-hot-continuous-v1_MainThread_ROOT INFO:Updating Building model Site:Location and SizingPeriod:DesignDay(s) to weather and ddy file...
[2023-06-28 08:54:00,264] EPLUS_ENV_5Zone-hot-continuous-v1_MainThread_ROOT INFO:Updating building model OutPut:Variable and variables XML tree model for BVCTB connection.
[2023-06-28 08:54:00,265] EPLUS_ENV_5Zone-hot-continuous-v1_MainThread_ROOT INFO:Setting up extra configuration in building model if exists...
[2023-06-28 08:54:00,265] EPLUS_ENV_5Zone-hot-continuous-v1_MainThread_ROOT INFO:Setting up action definition in building model if exists...


For the Rule-base controller have a look at the already defined controllers, there is one for each building, since the demo is based on the 5Zone building we are extending that controller and defining the action function we desire, feel free to play with the function to define your own action.

In [4]:
from sinergym.utils.controllers import RBC5Zone

class MyRuleBasedController(RBC5Zone):

    def act(self, observation: List[Any]) -> Sequence[Any]:
        """Select action based on outdoor air drybulb temperature and daytime.

        Args:
            observation (List[Any]): Perceived observation.

        Returns:
            Sequence[Any]: Action chosen.
        """
        obs_dict = dict(zip(self.variables['observation'], observation))

        out_temp = obs_dict['Site Outdoor Air Drybulb Temperature(Environment)']

        day = int(obs_dict['day'])
        month = int(obs_dict['month'])
        hour = int(obs_dict['hour'])
        year = int(obs_dict['year'])

        summer_start_date = datetime(year, 6, 1)
        summer_final_date = datetime(year, 9, 30)

        current_dt = datetime(year, month, day)

        # Get season comfort range
        if current_dt >= summer_start_date and current_dt <= summer_final_date:
            season_comfort_range = self.setpoints_summer
        else:
            season_comfort_range = self.setpoints_summer
        season_comfort_range = get_season_comfort_range(1991,month, day)
        # Update setpoints
        in_temp = obs_dict['Zone Air Temperature(SPACE1-1)']

        current_heat_setpoint = obs_dict[
            'Zone Thermostat Heating Setpoint Temperature(SPACE1-1)']
        current_cool_setpoint = obs_dict[
            'Zone Thermostat Cooling Setpoint Temperature(SPACE1-1)']

        new_heat_setpoint = current_heat_setpoint
        new_cool_setpoint = current_cool_setpoint

        if in_temp < season_comfort_range[0]:
            new_heat_setpoint = current_heat_setpoint + 1
            new_cool_setpoint = current_cool_setpoint + 1
        elif in_temp > season_comfort_range[1]:
            new_cool_setpoint = current_cool_setpoint - 1
            new_heat_setpoint = current_heat_setpoint - 1

        #Clip setpoints to the action space
        if new_heat_setpoint>self.env.action_space.high[0]:
            new_heat_setpoint=self.env.action_space.high[0]
        if new_heat_setpoint<self.env.action_space.low[0]:
            new_heat_setpoint=self.env.action_space.low[0]
        if new_cool_setpoint>self.env.action_space.high[1]:
            new_cool_setpoint=self.env.action_space.high[1]
        if new_cool_setpoint<self.env.action_space.low[1]:
            new_cool_setpoint=self.env.action_space.low[1]

        action = (new_heat_setpoint, new_cool_setpoint)
        if current_dt.weekday() > 5 or hour in range(22, 6):
            #weekend or night
            action = (18.33, 23.33)

        return action

Now that we have our controller ready we can use it:

In [5]:

# create rule-based controller
agent = MyRuleBasedController(env)

for i in range(1):
    obs, info = env.reset()
    rewards = []
    terminated = False
    current_month = 0
while not terminated:
    action = agent.act(obs)
    obs, reward, terminated, truncated, info = env.step(action)
    rewards.append(reward)
    if info['month'] != current_month:  # display results every month
        current_month = info['month']
        print('Reward: ', sum(rewards), info)
print(
    'Episode ',
    i,
    'Mean reward: ',
    np.mean(rewards),
    'Cumulative reward: ',
    sum(rewards))

[2023-06-28 08:54:05,364] EPLUS_ENV_5Zone-hot-continuous-v1_MainThread_ROOT INFO:Creating new EnergyPlus simulation episode...
[2023-06-28 08:54:05,528] EPLUS_ENV_5Zone-hot-continuous-v1_MainThread_ROOT INFO:EnergyPlus working directory is in /workspaces/sinergym/examples/Eplus-env-5Zone-hot-continuous-v1-res2/Eplus-env-sub_run1


/usr/local/lib/python3.10/dist-packages/opyplus/weather_data/weather_data.py:493: FutureWarning: the 'line_terminator'' keyword is deprecated, use 'lineterminator' instead.
  epw_content = self._headers_to_epw(use_datetimes=use_datetimes) + df.to_csv(
/usr/local/lib/python3.10/dist-packages/gymnasium/spaces/box.py:230: UserWarning: WARN: Casting input x to numpy array.
  gym.logger.warn("Casting input x to numpy array.")


Reward:  -0.029673378872410956 {'timestep': 1, 'time_elapsed': 900, 'year': 1991, 'month': 1, 'day': 1, 'hour': 0, 'action': (21.0, 25.0), 'reward': -0.029673378872410956, 'reward_energy': -0.05934675774482191, 'reward_comfort': -0.0, 'total_energy': 593.4675774482191, 'abs_comfort': 0.0, 'temperatures': [20.85245579021927]}
Reward:  -4252.742575790403 {'timestep': 2976, 'time_elapsed': 2678400, 'year': 1991, 'month': 2, 'day': 1, 'hour': 0, 'action': (15.0, 22.5), 'reward': -0.00610792304119184, 'reward_energy': -0.01221584608238368, 'reward_comfort': -0.0, 'total_energy': 122.1584608238368, 'abs_comfort': 0.0, 'temperatures': [20.27698188359977]}
Reward:  -5383.33275512995 {'timestep': 5664, 'time_elapsed': 5097600, 'year': 1991, 'month': 3, 'day': 1, 'hour': 0, 'action': (21.0, 28.5), 'reward': -0.009516267880754955, 'reward_energy': -0.01903253576150991, 'reward_comfort': -0.0, 'total_energy': 190.3253576150991, 'abs_comfort': 0.0, 'temperatures': [20.7657209636087]}
Reward:  -8358

Always remember to close the environment:

In [6]:
env.close()

[2023-06-28 08:54:25,062] EPLUS_ENV_5Zone-hot-continuous-v1_MainThread_ROOT INFO:EnergyPlus simulation closed successfully. 


.. note:: For more information about our defines controllers and how create a new one, please, visit our [Controller Documentation](https://ugr-sail.github.io/sinergym/compilation/html/pages/controllers.html)